<h1 align=center><font size = 5> Segmenting and Clustering Neighborhoods in Toronto</h1>

### We are required to:

1. Explore, 
2. Segment, 
3. Cluster the neighborhoods 

in the city of Toronto based on the postalcode and borough information.

<p>however, since the data is not readily available we have to web scrap a wikipedia page that contains this information</p>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

<h2 align=center><font size = 5>Part 1 - Webscrapping and storing data in dataframe</h2>

In [19]:
#conda install bs4

In [ ]:
#!conda install -c conda-forge geopy --yes

In [21]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [12]:
# importing all the necessary libraries
import numpy as np 
import pandas as pd 
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors 
from sklearn.cluster import KMeans 
from bs4 import BeautifulSoup 
from geopy.geocoders import Nominatim  
import folium 
print("Libraries imported.")

Libraries imported.


In [13]:
#our data lives in the following url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [14]:
#fetchig the page data using requests and storing it in a variable data
data = requests.get(url).text

In [15]:
#creating a Beautifulsoup objet
soup = BeautifulSoup(data, 'html.parser')

#checking the data and as it is vey long i have commented this line - feel free to uncommoent and check the data
#print(soup.prettify())

Creating empty list to create dataframes

In [16]:
neighborhood_table=[]

Searching and finding table and table data

In [17]:
table=soup.find('table')

1. Looping through the tables and looking for td tags, 
2. creating and initialising a dictionary that will hold  3 keys PostalCode, Borough and Neighborhood
3. If there is any row that has Not assigned we skip, 
4. PostCode : extracting postcode using row.p.text as it contains only 3 characters
5. Cleaning data in Borough and Neighborhood by remobing unwanted chracters using split, strip and replace functions
6. Finally append cleaned data to the neighborhood_table list

In [18]:

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        neighborhood_table.append(cell)

Creating dataframe with neighborhood_table list

In [19]:
#print(table_contents)
df=pd.DataFrame(neighborhood_table)

Checking our first 5 rows

In [20]:
df.head()

AttributeError: 'NoneType' object has no attribute 'items'

  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

### Printing the number of rows of your dataframe.

In [12]:
df.shape

(103, 3)

<h2 align=center><font size = 5>Part 2: Get the latitude and the longitude </h2>

In [19]:
#!conda install -c conda-forge geocoder --yes

### Download the csv file

In [13]:
url = 'http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
doc = requests.get(url)
with open('Geospatial_Coordinates.csv', 'wb') as f:
    f.write(doc.content)

### Using pandas to read and store the downloaded Geospacial_Coordinates.csv to get the coordinate for each postal code in a dataframe

In [14]:
df_coord = pd.read_csv('./Geospatial_Coordinates.csv')

### Let's check the first 5 rows

In [15]:
low_memory=False
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging the two dataframes (the df from part one and df_coord from part 2)

In [16]:
df_toronto = pd.merge(df, df_coord, how='left', left_on = 'PostalCode', right_on = 'Postal Code')

### Reading the first 5 rows

In [17]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


### Noticing that there are two columns with postal code, we need to remove the one after Neighborhood

In [18]:
df_toronto.drop('Postal Code', axis=1, inplace=True)

## Final result

In [19]:
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h2 align=center><font size = 5>Part 3: Explore and cluster the neighborhoods in Toronto.</h2>

#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>Toronto_explorer</em>, as shown below.

In [20]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer,")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Totonto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Totonto are 43.6534817, -79.3839347.


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [21]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


#### Create a map - using Folium python library  -of Toronto with neighborhoods superimposed on top.

In [22]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

let's simplify the above map and segment and cluster only boroughs that contain the word Toronto

In [23]:
toronto_data = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


We viualise the above

In [24]:
# create map of Manhattan using latitude and longitude values
map_toronto_neighborhood = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighborhood)  
    
map_toronto_neighborhood

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them

#### Define Foursquare Credentials and Version 
Please note this will be hidden for security reasons


In [68]:
#pip install sodapy

In [26]:
#import config
from sodapy import Socrata

In [64]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the first neighborhood in our dataframe.


In [30]:
toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [31]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [66]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


Let' check the results of the found url

In [67]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '605e35a0b9c0793689f7301d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

# function that extracts the category of the venue

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### clean the json and structure it into a pandas dataframe.

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Body Blitz Spa East,Spa,43.654735,-79.359874


Above are only the first 5 venues , let's see how many are in total

In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


### Exploring neighborhoods in a part of Toronto City:

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Getting nearby venues for each neighborhood

In [38]:
nearby_venues_for_each_neighborhood = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [39]:
print(nearby_venues_for_each_neighborhood.shape)
nearby_venues_for_each_neighborhood.head()

(1586, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Let's check the size of the resulting dataframe

In [40]:
nearby_venues_for_each_neighborhood.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,65,65,65,65,65,65
Christie,15,15,15,15,15,15
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36
Davisville North,8,8,8,8,8,8


### Number of distinct categories

In [41]:
print('{} uniques categories.'.format(len(nearby_venues_for_each_neighborhood['Venue Category'].unique())))

235 uniques categories.


## Analyze Each Neighborhood

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(nearby_venues_for_each_neighborhood[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = nearby_venues_for_each_neighborhood['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Examine the new dataframe size

In [43]:
toronto_onehot.shape

(1586, 235)

### Creating dataframe where venues are grouped by frequency for each Neighborhood

In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.071429,0.142857,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,Central Bay Street,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015385,0.0,0.0,0.015385,0.0
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


In [45]:
toronto_grouped.shape # confirming the new size

(39, 235)

### Retrieving 7 most common venues for each Neighborhood

### Detailed  

In [46]:
num_top_venues = 7

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3            Pharmacy  0.03
4  Seafood Restaurant  0.03
5         Cheese Shop  0.03
6      Farmers Market  0.03


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.13
1         Breakfast Spot  0.09
2            Coffee Shop  0.09
3                 Bakery  0.04
4              Pet Store  0.04
5  Performing Arts Venue  0.04
6      Convenience Store  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.14
1     Airport Terminal  0.14
2      Harbor / Marina  0.07
3              Airport  0.07
4  Rental Car Location  0.07
5          Coffee Shop  0.07
6     Sculpture Garden  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1                

### Summary of the same results in a matrix

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Seafood Restaurant,Cheese Shop,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Pet Store,Performing Arts Venue,Convenience Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Harbor / Marina,Airport,Rental Car Location,Coffee Shop,Sculpture Garden
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Thai Restaurant,Japanese Restaurant
4,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Restaurant


## Clustering Neighborhood using K-Means:

Run _k_-means to cluster the neighborhood into 5 clusters.

In [48]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

## clustering top 7 venues for each Neighborhood and creating a dataframe

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged_ = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged_ = toronto_merged_.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,1,1,Coffee Shop,Pub,Park,Bakery,Café,Theater,Breakfast Spot
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,1,1,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,1,1,Café,Coffee Shop,Gastropub,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,1,1,Health Food Store,Trail,Pub,Yoga Studio,Music Venue,Mediterranean Restaurant,Men's Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,1,1,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Seafood Restaurant,Cheese Shop,Farmers Market


Finally, let's visualize the resulting clusters using Foliom

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_['Latitude'], toronto_merged_['Longitude'], toronto_merged_['Neighborhood'], toronto_merged_['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

We Can now examine each cluster and determine the discriminating venue categories that distinguish each cluste

#### Cluster 1

In [56]:
toronto_merged_.loc[toronto_merged_['Cluster Labels'] == 0, toronto_merged_.columns[[1] + list(range(5, toronto_merged_.shape[1]))]]

,Borough,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
9,East YorkEast Toronto,0,0,0,Convenience Store,Park,Coffee Shop,Intersection,New American Restaurant,Men's Store,Mexican Restaurant
18,Central Toronto,0,0,0,Park,Bus Line,Dim Sum Restaurant,Swim School,Yoga Studio,Music Venue,Men's Store
33,Downtown Toronto,0,0,0,Park,Playground,Trail,Music Venue,Massage Studio,Mediterranean Restaurant,Men's Store


#### Cluster 2

In [57]:
toronto_merged_.loc[toronto_merged_['Cluster Labels'] == 1, toronto_merged_.columns[[1] + list(range(5, toronto_merged_.shape[1]))]]

,Borough,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Downtown Toronto,1,1,1,Coffee Shop,Pub,Park,Bakery,Café,Theater,Breakfast Spot
1,Downtown Toronto,1,1,1,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop
2,Downtown Toronto,1,1,1,Café,Coffee Shop,Gastropub,Cocktail Bar,Cosmetics Shop,Italian Restaurant,Restaurant
3,East Toronto,1,1,1,Health Food Store,Trail,Pub,Yoga Studio,Music Venue,Mediterranean Restaurant,Men's Store
4,Downtown Toronto,1,1,1,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Seafood Restaurant,Cheese Shop,Farmers Market
5,Downtown Toronto,1,1,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Burger Joint,Thai Restaurant,Japanese Restaurant
6,Downtown Toronto,1,1,1,Grocery Store,Café,Park,Baby Store,Coffee Shop,Italian Restaurant,Restaurant
7,Downtown Toronto,1,1,1,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Hotel,Clothing Store
8,West Toronto,1,1,1,Pharmacy,Bakery,Pool,Brewery,Middle Eastern Restaurant,Café,Supermarket
10,Downtown Toronto,1,1,1,Coffee Shop,Aquarium,Café,Hotel,Brewery,Fried Chicken Joint,Sporting Goods Shop


#### Cluster 3

In [58]:
toronto_merged_.loc[toronto_merged_['Cluster Labels'] == 2, toronto_merged_.columns[[1] + list(range(5, toronto_merged_.shape[1]))]]

,Borough,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
29,Central Toronto,2,2,2,Playground,Tennis Court,Restaurant,Gym,Pet Store,Music Venue,Mediterranean Restaurant


#### Cluster 4

In [59]:
toronto_merged_.loc[toronto_merged_['Cluster Labels'] ==3, toronto_merged_.columns[[1] + list(range(5, toronto_merged_.shape[1]))]]

,Borough,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
19,Central Toronto,3,3,3,Garden,Home Service,New American Restaurant,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


#### Cluster 5

In [60]:
toronto_merged_.loc[toronto_merged_['Cluster Labels'] == 4, toronto_merged_.columns[[1] + list(range(5, toronto_merged_.shape[1]))]]

,Borough,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
21,Central Toronto,4,4,4,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Museum,Massage Studio


#### Cluster 6

In [61]:
toronto_merged_.loc[toronto_merged_['Cluster Labels'] == 5, toronto_merged_.columns[[1] + list(range(5, toronto_merged_.shape[1]))]]

,Borough,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue


#### Cluster 7

In [62]:
toronto_merged_.loc[toronto_merged_['Cluster Labels'] == 6, toronto_merged_.columns[[1] + list(range(5, toronto_merged_.shape[1]))]]

,Borough,Cluster Label,Cluster Labelsz,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
